# Running Genetic Algorithm

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from copy import deepcopy
from functools import partial
import os
from itertools import product
import json


In [26]:
import sys

sys.path.append("..")

from functions.utils import *
from functions.crossover import *
from functions.selection_algos import *
from functions.mutations import *
from functions.algorithms import *
from functions.solutions import *

In [27]:
scores = pd.read_csv('data/seating_data(in).csv', index_col=0)
scores_array = scores.to_numpy()
scores_array

array([[   0, 5000,    0, ...,    0,    0,    0],
       [5000,    0,  700, ...,    0,    0,    0],
       [   0,  700,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,  700,  700],
       [   0,    0,    0, ...,  700,    0,  900],
       [   0,    0,    0, ...,  700,  900,    0]])

# Grid search 

In [28]:
pop_sizes = [50] #10
max_gens = [100] 

mutation_ops = [swap_mutation, scramble_mutation_optimized, inversion_mutation]
crossover_ops = [group_preserving_order_crossover, pmx_table_block_crossover]
selection_names = ["tournament", "rank"]  
tourn_ks = [3, 5, 7, 10, 15] 
rank_ls = [0.1, 1, 10] # 0.1 - linear 
xo_probs = [0.6, 0.7, 0.8, 0.9, 1]  
mut_probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] 
elitisms = [True, False] 


> We will load the result from the CSV file to store the tested configurations and save those in a set of tuples (tested_configs). 

In [29]:
results_file = "results/ga_results.csv"

In [30]:
# Load or initialize the results dataframe
if os.path.exists(results_file):
    df_existing = pd.read_csv(results_file)
    tested_configs = set(config_to_key(row) for _, row in df_existing.iterrows())
else:
    df_existing = pd.DataFrame()
    tested_configs = set()

> In the grid search loop, we will iterate over all combinations of the parameters. <br>
Each combination of parameters will be ran 30 times to obtain a large enough sample size to compare the results. <br> 
The results will be saved in a CSV file, so that we can load it later and append new results.<br>
If combination of parameters has already been tested, we will skip it, allowing efficient exploration of the hyperparameter space.

In [31]:
for pop_size, mutation_fn, crossover_fn, selection_name, max_gen, elitism, xo_prob, mut_prob in product(
    pop_sizes, mutation_ops, crossover_ops, selection_names, max_gens, elitisms, xo_probs, mut_probs
):

    # Skip if incompatible selection and missing param
    if selection_name == "tournament":
        selection_configs = tourn_ks
    elif selection_name == "rank":
        selection_configs = rank_ls
    else:
        selection_configs = [None]  # fitness has no extra param

    for sel_config in selection_configs:
        # Create a tuple key for the current configurations 
        
        if mutation_fn.__name__ == "scramble_mutation_optimized":
            mutation_name = "scramble_mutation"
        else:
            mutation_name = mutation_fn.__name__
        current_key = (
            pop_size,
            mutation_name,
            crossover_fn.__name__,
            selection_name,
            sel_config,
            elitism,
            max_gen,
            round(xo_prob, 4),
            round(mut_prob, 4)
        )
        
        # If the current configuration has already been tested (in tested_configs), skip it
        if current_key in tested_configs:
            print(f"Skipping already tested config: {current_key}")
            continue
        
        # Set selection algorithm hyperparameters
        if selection_name == "tournament":
            selection_algorithm = partial(tournament_selection_optimized, k=sel_config)
        else:  # rank
            selection_algorithm = partial(rank_selection_optimized, function="exponential", l=sel_config)

        # Repeat GA run 30 times for each configuration
        fitness_scores = []
        conv_gens = []
        conv_times = []
        
        print(f"Running GA with config: {current_key}")
        for _ in range(30):
            population = [
                Wedding_GA_Solution(
                    scores=scores_array,
                    mutation_function=mutation_fn,
                    crossover_function=crossover_fn
                ) for _ in range(pop_size)
            ]

            best_sol, _, conv_gen, conv_time = genetic_algorithm_optimized(
                initial_population=deepcopy(population),
                max_gen=max_gen,
                selection_algorithm=selection_algorithm,
                maximization=True,
                xo_prob=xo_prob,
                mut_prob=mut_prob,
                elitism=elitism,
                verbose=False
            )

            fitness_scores.append(best_sol.fitness())
            conv_gens.append(conv_gen)
            conv_times.append(round(conv_time, 2))
            
        avg_fitness = round(np.mean(fitness_scores), 2)
        std_fitness = round(np.std(fitness_scores),2)
        
        avg_conv_gen = round(np.mean(conv_gens),2)
        avg_conv_time = round(np.mean(conv_times),2)
        
        print(f"Avg fitness: {round(avg_fitness, 1)}, Std: {round(std_fitness,1)} in {avg_conv_gen} generations, {avg_conv_time} seconds")
        
        df_new = pd.DataFrame([{
            "pop_size": pop_size,
            "mutation": mutation_fn.__name__,
            "crossover": crossover_fn.__name__,
            "selection": selection_name,
            "selection_param": sel_config,
            "elitism": elitism,
            "max_gen": max_gen,
            "xo_prob": round(xo_prob, 4),
            "mut_prob": round(mut_prob, 4),
            "avg_fitness": avg_fitness,
            "std_fitness": std_fitness,
            "avg_conv_gen": avg_conv_gen,
            "avg_conv_time": avg_conv_time,
            "fitness_scores": json.dumps([float(x) for x in fitness_scores]),
        }])
        
        # Append to existing CSV (or write new one)
        if os.path.exists(results_file) and not df_new.empty:
            # mode = "a" = append
            df_new.to_csv(results_file, mode='a', header=False, index=False) 
        elif not df_new.empty:
            df_new.to_csv(results_file, index=False)
        
        tested_configs.add(current_key)

Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 3, True, 100, 0.6, 0.1)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 5, True, 100, 0.6, 0.1)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 7, True, 100, 0.6, 0.1)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 10, True, 100, 0.6, 0.1)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 15, True, 100, 0.6, 0.1)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 3, True, 100, 0.6, 0.2)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament', 5, True, 100, 0.6, 0.2)
Skipping already tested config: (50, 'swap_mutation', 'group_preserving_order_crossover', 'tournament'

# Evaluate top 10 configurations and top configuration for each operator

> We will run again with 100 episodes / runs for: 
> - the top 10 configurations (sorted by medial final fitness) and,
> - the best configuration for each mutation, crossover and selection operator 

> We want to achieve higher confidence in the results<br>
We will also run for more generations (200)

In [32]:
df = pd.read_csv("results/ga_results.csv", index_col=0)

In [33]:
df["fitness_list"] = df["fitness_scores"].apply(json.loads)
df["fitness_median"] = df["fitness_list"].apply(np.median)
df = df.sort_values(by=["fitness_median"], ascending=False)
df.head(10)

,pop_size,mutation,crossover,selection,selection_param,elitism,max_gen,xo_prob,mut_prob,avg_fitness,std_fitness,avg_conv_gen,avg_conv_time,fitness_scores,fitness_list,fitness_median
3467,50,swap_mutation,pmx_table_block_crossover,rank,10.0,False,100,0.6,1.0,75050.0,2274.02,83.73,1.01,"[75900.0, 73600.0, 74300.0, 73400.0, 75900.0, ...","[75900.0, 73600.0, 74300.0, 73400.0, 75900.0, ...",75450.0
1395,50,swap_mutation,pmx_table_block_crossover,rank,1.0,True,100,0.9,0.9,74710.0,2788.83,88.50,1.15,"[75100.0, 77200.0, 74700.0, 76300.0, 72300.0, ...","[75100.0, 77200.0, 74700.0, 76300.0, 72300.0, ...",75400.0
1209,50,swap_mutation,pmx_table_block_crossover,rank,1.0,False,100,0.8,0.8,74680.0,2190.19,88.67,1.06,"[77700.0, 71300.0, 77000.0, 72400.0, 75400.0, ...","[77700.0, 71300.0, 77000.0, 72400.0, 75400.0, ...",75350.0
1876,50,swap_mutation,pmx_table_block_crossover,rank,1.0,False,100,0.7,0.9,74650.0,2616.58,89.07,1.11,"[75300.0, 78700.0, 72400.0, 75500.0, 73800.0, ...","[75300.0, 78700.0, 72400.0, 75500.0, 73800.0, ...",75200.0
3431,50,swap_mutation,pmx_table_block_crossover,rank,10.0,True,100,0.8,1.0,74920.0,2488.16,84.87,1.05,"[75400.0, 74800.0, 73100.0, 75800.0, 77600.0, ...","[75400.0, 74800.0, 73100.0, 75800.0, 77600.0, ...",75150.0
1860,50,swap_mutation,pmx_table_block_crossover,rank,10.0,False,100,0.6,0.7,73770.0,3236.68,88.17,1.06,"[74800.0, 66700.0, 67400.0, 76900.0, 76300.0, ...","[74800.0, 66700.0, 67400.0, 76900.0, 76300.0, ...",75050.0
3428,50,swap_mutation,pmx_table_block_crossover,rank,10.0,True,100,0.7,1.0,74930.0,2162.43,86.10,1.05,"[75100.0, 77700.0, 73200.0, 76800.0, 75700.0, ...","[75100.0, 77700.0, 73200.0, 76800.0, 75700.0, ...",75050.0
3433,50,swap_mutation,pmx_table_block_crossover,rank,1.0,True,100,0.9,1.0,74690.0,2501.51,91.87,1.30,"[75500.0, 75000.0, 73500.0, 76400.0, 76200.0, ...","[75500.0, 75000.0, 73500.0, 76400.0, 76200.0, ...",75050.0
1840,50,swap_mutation,pmx_table_block_crossover,rank,10.0,True,100,0.8,0.9,74390.0,3055.20,86.77,1.16,"[74800.0, 75900.0, 74400.0, 71400.0, 72500.0, ...","[74800.0, 75900.0, 74400.0, 71400.0, 72500.0, ...",75000.0
3425,50,swap_mutation,pmx_table_block_crossover,rank,10.0,True,100,0.6,1.0,74690.0,2482.92,85.67,1.03,"[72300.0, 74700.0, 75400.0, 74800.0, 74000.0, ...","[72300.0, 74700.0, 75400.0, 74800.0, 74000.0, ...",75000.0


> We will save this new results in "avg_fitness_by_generation.csv"

In [34]:
mutation_ops = {
    "swap_mutation": swap_mutation,
    "scramble_mutation": scramble_mutation_optimized,
    "inversion_mutation": inversion_mutation,
}
crossover_ops = {
    "group_preserving_order_crossover": group_preserving_order_crossover,
    "pmx_table_block_crossover": pmx_table_block_crossover,
}

results_file = "results/avg_fitness_by_generation.csv"

num_runs = 100
max_gen = 200
pop_size = 50


top_configs = df.head(10)
top_by_mutation = df.loc[df.groupby("mutation")["fitness_median"].idxmax()]

top_by_crossover = df.loc[df.groupby("crossover")["fitness_median"].idxmax()]

top_by_selection = df.loc[df.groupby("selection")["fitness_median"].idxmax()]

top_all = pd.concat([
    top_by_mutation.assign(group="mutation"),
    top_by_crossover.assign(group="crossover"),
    top_by_selection.assign(group="selection")
])

top_all = pd.concat([top_all, top_configs])

scores = pd.read_csv('data/seating_data(in).csv', index_col=0)
scores_array = scores.to_numpy()

if os.path.exists(results_file):
    df_existing = pd.read_csv(results_file)
    tested_configs = set(row["configuration"] for _, row in df_existing.iterrows())
else:
    df_existing = pd.DataFrame()
    tested_configs = set()
    

for _, row in top_all.iterrows():
    config_key = (
        f"{row['mutation']} | {row['crossover']} | "
        f"{row['selection']}({row['selection_param']}) | "
        f"xo={row['xo_prob']} | mut={row['mut_prob']} | elitism={row['elitism']}"
    )
    
    if config_key in tested_configs:
        print(f"Skipping tested config: {config_key}")
        continue

    if row["selection"] == "tournament":
        selection_fn = partial(tournament_selection_optimized, k=int(row["selection_param"]))
    elif row["selection"] == "rank":
        selection_fn = partial(rank_selection_optimized, function="exponential", l=row["selection_param"])

    mutation_fn = mutation_ops[row["mutation"]]
    crossover_fn = crossover_ops[row["crossover"]]

    fitness_gen_all = []
    
    print(f"Running GA with config: ({config_key}) with {num_runs} runs")
    for _ in range(num_runs):
        population = [
            Wedding_GA_Solution(
                scores=scores_array,
                mutation_function=mutation_fn,
                crossover_function=crossover_fn,
            ) for _ in range(pop_size)
        ]

        _, fitness_hist, _, _ = genetic_algorithm_optimized(
            initial_population=population,
            selection_algorithm=selection_fn,
            max_gen=max_gen,
            maximization=True,
            elitism=row["elitism"],
            xo_prob=row["xo_prob"],
            mut_prob=row["mut_prob"],
            verbose=False,
        )

        fitness_gen_all.append(fitness_hist)

    median_fitness_by_gen = np.median(fitness_gen_all, axis=0)

    df_new = pd.DataFrame([{
        "configuration": config_key,
        "fitness_per_generation": json.dumps(list(median_fitness_by_gen))
    }])

    if os.path.exists(results_file) and not df_new.empty:
        df_new.to_csv(results_file, mode='a', header=False, index=False)
    else:
        df_new.to_csv(results_file, index=False)
        
    tested_configs.add(config_key)

Skipping tested config: inversion_mutation | pmx_table_block_crossover | tournament(10.0) | xo=1.0 | mut=0.7 | elitism=False
Skipping tested config: scramble_mutation | group_preserving_order_crossover | rank(10.0) | xo=0.6 | mut=1.0 | elitism=True
Skipping tested config: swap_mutation | pmx_table_block_crossover | rank(10.0) | xo=0.6 | mut=1.0 | elitism=False
Skipping tested config: swap_mutation | group_preserving_order_crossover | rank(10.0) | xo=0.8 | mut=1.0 | elitism=True
Skipping tested config: swap_mutation | pmx_table_block_crossover | rank(10.0) | xo=0.6 | mut=1.0 | elitism=False
Skipping tested config: swap_mutation | pmx_table_block_crossover | rank(10.0) | xo=0.6 | mut=1.0 | elitism=False
Skipping tested config: swap_mutation | pmx_table_block_crossover | tournament(15.0) | xo=1.0 | mut=0.7 | elitism=False
Skipping tested config: swap_mutation | pmx_table_block_crossover | rank(10.0) | xo=0.6 | mut=1.0 | elitism=False
Skipping tested config: swap_mutation | pmx_table_block